In [1]:
from os.path import join, expanduser
from emit_tools import load_EMIT_swath, extract_GLT, apply_GLT, ortho_xr

In [2]:
filename = expanduser(join("~", "Downloads", "EMIT_L2A_RFL_001_20240521T232728_2414216_003.nc"))

In [3]:
swath_ds = load_EMIT_swath(filename)
swath_ds

<xarray.Dataset> Size: 2GB
Dimensions:           (downtrack: 1280, crosstrack: 1242, wavelengths: 285,
                       ortho_y: 1893, ortho_x: 2327)
Coordinates:
  * downtrack         (downtrack) int64 10kB 0 1 2 3 4 ... 1276 1277 1278 1279
  * crosstrack        (crosstrack) int64 10kB 0 1 2 3 4 ... 1238 1239 1240 1241
    lon               (downtrack, crosstrack) float64 13MB ...
    lat               (downtrack, crosstrack) float64 13MB ...
    elev              (downtrack, crosstrack) float64 13MB ...
    glt_x             (ortho_y, ortho_x) float64 35MB ...
    glt_y             (ortho_y, ortho_x) float64 35MB ...
  * wavelengths       (wavelengths) float32 1kB 381.0 388.4 ... 2.493e+03
    fwhm              (wavelengths) float32 1kB ...
    good_wavelengths  (wavelengths) float32 1kB ...
Dimensions without coordinates: ortho_y, ortho_x
Data variables:
    reflectance       (downtrack, crosstrack, wavelengths) float32 2GB ...
Attributes: (12/39)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatialResolution:                 0.000542232520256367
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-1.16103646e+02  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20240521T232728_24142...

In [4]:
GLT_array = extract_GLT(swath_ds)
GLT_array

array([[[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       ...,

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]]])

In [5]:
type(GLT_array)

numpy.ndarray

In [6]:
GLT_array.shape

(1893, 2327, 2)

In [7]:
ortho_ds = ortho_xr(swath_ds)
ortho_ds

<xarray.Dataset> Size: 5GB
Dimensions:           (latitude: 1893, longitude: 2327, wavelengths: 285)
Coordinates:
  * wavelengths       (wavelengths) float32 1kB 381.0 388.4 ... 2.493e+03
    fwhm              (wavelengths) float32 1kB ...
    good_wavelengths  (wavelengths) float32 1kB ...
  * latitude          (latitude) float64 15kB 33.95 33.95 33.95 ... 32.93 32.93
  * longitude         (longitude) float64 19kB -116.1 -116.1 ... -114.8 -114.8
    elev              (latitude, longitude) float32 18MB -9.999e+03 ... -9.99...
    spatial_ref       int64 8B 0
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 5GB -9.999e+...
Attributes: (12/39)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatialResolution:                 0.000542232520256367
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-1.16103646e+02  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20240521T232728_24142...

In [8]:
swath_array = swath_ds["reflectance"].data
swath_array

array([[[0.04310642, 0.04336452, 0.04150592, ..., 0.10212765,
         0.0974834 , 0.09012822],
        [0.06145171, 0.06501866, 0.06859049, ..., 0.10978349,
         0.10137388, 0.09884916],
        [0.06124046, 0.0649093 , 0.0685825 , ..., 0.10782409,
         0.09501854, 0.0907679 ],
        ...,
        [0.00507217, 0.00802279, 0.01125222, ..., 0.1837036 ,
         0.18001686, 0.17860676],
        [0.0060593 , 0.00890038, 0.00998863, ..., 0.19765685,
         0.19258572, 0.18802734],
        [0.01689515, 0.01768825, 0.01971469, ..., 0.18779285,
         0.17850284, 0.17630193]],

       [[0.05653568, 0.06005755, 0.06357399, ..., 0.10565821,
         0.10205421, 0.09661389],
        [0.06207323, 0.06573673, 0.0694032 , ..., 0.10960843,
         0.09955885, 0.09995052],
        [0.06266887, 0.06635007, 0.07002756, ..., 0.12251341,
         0.10631712, 0.10210592],
        ...,
        [0.0090206 , 0.01041509, 0.01469167, ..., 0.19406983,
         0.18753886, 0.18678044],
        [0.0

In [9]:
swath_array.shape

(1280, 1242, 285)

In [10]:
type(swath_array)

numpy.ndarray

In [13]:
ortho_array = apply_GLT(swath_array, GLT_array)
ortho_array

array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-99

In [14]:
type(ortho_array)

numpy.ndarray

In [15]:
ortho_array.shape

(1893, 2327, 285)